# Creating and accessing SQL databases with python using sqlalchemy and sqlite

This post includes code adapted from [SQLAlchemy Example](https://gist.github.com/ronreiter/98916876e14534fa8ab2).

In [1]:
import sqlalchemy as db
import sqlite3
import pandas as pd

In [2]:
from sqlalchemy import Column, Integer, String, ForeignKey, create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref, sessionmaker, joinedload

# For this example we will use an in-memory sqlite DB.
# Let's also configure it to echo everything it does to the screen.
engine = create_engine('sqlite:///:memory:', echo=True)

In [3]:
# The base class which our objects will be defined on.
Base = declarative_base()

# Our User object, mapped to the 'users' table
class User(Base):
    __tablename__ = 'users'

    # Every SQLAlchemy table should have a primary key named 'id'
    id = Column(Integer, primary_key=True)

    name = Column(String)
    fullname = Column(String)
    password = Column(String)

    # Lets us print out a user object conveniently.
    def __repr__(self):
       return "<User(name='%s', fullname='%s', password'%s')>" % (
                               self.name, self.fullname, self.password)

In [4]:
# The Address object stores the addresses 
# of a user in the 'adressess' table.
class Address(Base):
    __tablename__ = 'addresses'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)

    # Since we have a 1:n relationship, we need to store a foreign key 
    # to the users table.
    user_id = Column(Integer, ForeignKey('users.id'))

    # Defines the 1:n relationship between users and addresses.
    # Also creates a backreference which is accessible from a User object.
    user = relationship("User", backref=backref('addresses'))

    # Lets us print out an address object conveniently.
    def __repr__(self):
        return "<Address(email_address='%s')>" % self.email_address

In [6]:

# Create all tables by issuing CREATE TABLE commands to the DB.
Base.metadata.create_all(engine) 

# Creates a new session to the database by using the engine we described.
Session = sessionmaker(bind=engine)
session = Session()

# Let's create a user and add two e-mail addresses to that user.
example_user = User(name='example', fullname='example last_name_example', password='examplepassword')
example_user.addresses = [Address(email_address='example@gmail.com'), Address(email_address='example@yahoo.com')]

# Let's add the user and its addresses we've created to the DB and commit.
session.add(example_user)
session.commit()

# Now let's query the user that has the e-mail address ed@google.com
# SQLAlchemy will construct a JOIN query automatically.
user_by_email = session.query(User)\
    .filter(Address.email_address=='example@gmail.com')\
    .first()

2020-10-02 08:55:48,507 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2020-10-02 08:55:48,508 INFO sqlalchemy.engine.base.Engine ()
2020-10-02 08:55:48,510 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("addresses")
2020-10-02 08:55:48,510 INFO sqlalchemy.engine.base.Engine ()
2020-10-02 08:55:48,513 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-10-02 08:55:48,514 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2020-10-02 08:55:48,515 INFO sqlalchemy.engine.base.Engine ('example', 'example last_name_example', 'examplepassword')
2020-10-02 08:55:48,517 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2020-10-02 08:55:48,517 INFO sqlalchemy.engine.base.Engine ('example@gmail.com', 1)
2020-10-02 08:55:48,519 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2020-10-02 08:55:48,519 INFO sqlalchemy.engine.bas

In [7]:
print(user_by_email)

<User(name='example', fullname='example last_name_example', password'examplepassword')>


In [8]:
# This will cause an additional query by lazy loading from the DB.
print(user_by_email.addresses)

2020-10-02 08:55:53,081 INFO sqlalchemy.engine.base.Engine SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses 
WHERE ? = addresses.user_id
2020-10-02 08:55:53,083 INFO sqlalchemy.engine.base.Engine (1,)
[<Address(email_address='example@gmail.com')>, <Address(email_address='example@yahoo.com')>]


In [10]:
# To avoid querying again when getting all addresses of a user,
# we use the joinedload option. SQLAlchemy will load all results and hide
# the duplicate entries from us, so we can then get for
# the user's addressess without an additional query to the DB.
user_by_email = session.query(User)\
    .filter(Address.email_address=='example@gmail.com')\
    .options(joinedload(User.addresses))\
    .first()

2020-10-02 08:56:04,305 INFO sqlalchemy.engine.base.Engine SELECT anon_1.users_id AS anon_1_users_id, anon_1.users_name AS anon_1_users_name, anon_1.users_fullname AS anon_1_users_fullname, anon_1.users_password AS anon_1_users_password, addresses_1.id AS addresses_1_id, addresses_1.email_address AS addresses_1_email_address, addresses_1.user_id AS addresses_1_user_id 
FROM (SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users, addresses 
WHERE addresses.email_address = ?
 LIMIT ? OFFSET ?) AS anon_1 LEFT OUTER JOIN addresses AS addresses_1 ON anon_1.users_id = addresses_1.user_id
2020-10-02 08:56:04,306 INFO sqlalchemy.engine.base.Engine ('example@gmail.com', 1, 0)


In [11]:
print(user_by_email)

<User(name='example', fullname='example last_name_example', password'examplepassword')>


In [12]:
print(user_by_email.addresses)

[<Address(email_address='example@gmail.com')>, <Address(email_address='example@yahoo.com')>]
